# Companies from 1994 to 2018
This notebook contains cells with code that web scrapes data from SEC archive of historical documents (https://www.sec.gov/cgi-bin/srch-edgar), then creates the table with 5 columns:

- Company: name of the company
- CIK: Central Index Key
- Form: type of form (S-1, S-1/A, S-1MEF
- Filling date: date of filling
- URL: link of the document

In [1]:
# import our libraries
import requests
import time
import datetime
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

## exp_seq_merge_int()
Function that converts any long-written integer into list.
For example, I have integer 123456789101112, the exp_seq_merge_int would convert it into list [1,2,3,4,5,6,7,8,9,10,11,12]. 

It helps the code to correctly parse through right amount of pages per search

In [2]:
def exp_seq_merge_str(digits_str, base):
    combined_list = []
    merge_len = 1
    start_at = 0
    end_at = base
    while start_at < end_at:
        for i in range(start_at, end_at, merge_len):
            combined_str = ''
            for ii in range(i, i + merge_len):
                combined_str += digits_str[ii]
            combined_list.append(combined_str)
        start_at = end_at
        end_at *= base
        end_at = end_at if end_at < len(digits_str) else len(digits_str)
        merge_len += 1

    return combined_list

def exp_seq_merge_int(n, base):
    return list(map(int, exp_seq_merge_str('0'+str(n), base)))[1:]

## get_filing_link()

Function get_filing_link() retrives the URL of text document.

In [3]:
def get_filing_link(table_data):
    base_url = 'https://www.sec.gov/'
    td_href = table_data[comp_row][2].find('a', {'href':True})['href'] 
    url = base_url + td_href
    return url

## get_CIK()

Function get_CIK() scans the content of URL and parses the central index key inside the document.

In [4]:
# New
def get_CIK(url):
    headers = {"User-Agent": "My-User_Agent"}
    req = requests.get(url, headers={'User-Agent': 'My-User_Agent', 'From': 'tnurmanov@drew.edu'})
        
    try:
        contents = req.content.decode('utf-8').split()
        contents.index('KEY:')
        index = contents.index('KEY:')
        cik = contents[index+1]
    except ValueError:
        cik = 'NO CIK FOUND'
    except UnicodeDecodeError:
        cik = 'UnicodeError'
        
    return cik

## Main code

I ran this code three times. First it parsed data of years 1994 to 2002, 2003 to 2010, and 2011 to 2018. Every attempt to process the whole preiod (1994-2018) at once, ended up the code crash, sinc. Therefore it's safer way to divide the operation into tries.

The way I completed the operation:

1. Main code 
    - Assign variable 'year = [i for i in range(1994,2003)]' 
    - Run it
2. 1994-2002 
    - Save into excel file
3. Main code 
    - Assign variable 'year = [i for i in range(2003,2011)]' 
    - Run it
4. 2003-2010 
    - Save into excel file
5. Main code 
    - Assign variable 'year = [i for i in range(2011,2019)]' 
    - Run it
6. 2011-2018 
    - Save into excel file
7. Using Pandas 
    - Upload excel files back as df1, df2, df3
8. Using Pandas 
    - Remove extra value types from 'Form' column
9. Using Pandas 
    - Concantenate df1, df2, df3 into excel file separated by worksheets


In [5]:
# I wanted to see how long exactly it took me to process the code
startTime = time.time()

# First run, set sequnce 1994 to 2003.
# Second, 2003 to 2011
# Third 2011 to 2019
# Because of the Python's nature we need to end it by 2019, but last number is 2018
year = [i for i in range(2011,2019)]

# Dictionary to store companies
companies = {}

# Lists of a company CIK, type of form, date
company_name = []
company_cik = []
company_form = []
company_date = []
company_url = []

for j in range(len(year)):
    
    print('-'*50)
    print('Year:', str(year[j]), '---', 'Processing...')
    
    # base URL for the SEC EDGAR browser
    endpoint = r"https://www.sec.gov/cgi-bin/srch-edgar"

    ########################################################
    # This piece of code defines a number of pages per search
    # therefore computer 

    # define our parameters dictionary
    param_dict = {'text':'S-1 OR S-1/A OR S-1MEF',
                  'start':'1',
                  'count':'80',
                  'first':str(year[j]),
                  'last':str(year[j])}

    # request the url, and then parse the response.
    response = requests.get(url = endpoint, params = param_dict)
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the page table with our data 
    table = soup.find('div', attrs = {'style':'margin-left: 10px'})
    page_amount = table.findAll('center')[0]
    page_amount = page_amount.text
    page_amount = page_amount.replace(" ", "")
    page_amount = page_amount[1:-6]
    
    if len(page_amount) <= 9:
        base = len(page_amount)+1
    else:
        base = 10
        
    page_list = exp_seq_merge_int(page_amount, base)
    ########################################################
    
    page = 1
    page_info = 1
    
    # PAGE
    for i in range(len(page_list)):
        # define our parameters dictionary
        param_dict = {'text':'S-1 OR S-1/A OR S-1MEF',
                      'start':str(page),
                      'count':'80',
                      'first':str(year[j]),
                      'last':str(year[j])}
        
        print('Page:', page_info, 'out of', len(page_list), end='\r')
        
        # request the url, and then parse the response.
        response = requests.get(url = endpoint, params = param_dict)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # find the page table with our data 
        table = soup.find('div', attrs = {'style':'margin-left: 10px'})
        
        # List of given data of all companies
        table_data = []
        
        # This "for loop" gets rid of all attributes 
        # that don't relate to this task
        for row in table.findAll('tr'):
            data = row.findAll('td')
            # if attribute length is equal to 6, it will be added to table_data
            if len(data) == 6:
                table_data.append(data)
                
        
        # FILES
        comp_row = 0   
        
        for i in range(len(table_data)-1):
            url = get_filing_link(table_data)
            company_url.append(url)
            company_cik.append(get_CIK(url))
            company_name.append(table_data[comp_row][1].text)
            company_date.append(table_data[comp_row][4].text)
            company_form.append(table_data[comp_row][3].text)
            comp_row += 1
            
        page += 80
        page_info += 1
    
    # Last file has to be retrived outside the 'for' loop
    url = get_filing_link(table_data)
    company_url.append(url)
    company_cik.append(get_CIK(url))
    company_name.append(table_data[comp_row][1].text)
    company_date.append(table_data[comp_row][4].text)
    company_form.append(table_data[comp_row][3].text)
    
    print('Year:', str(year[j]), '---', 'Complete!')

companies['Company Name'] = company_name    
companies['CIK'] = company_cik
companies['Form'] = company_form
companies['Filing Date'] = company_date
companies['URL'] = company_url


print('\n')
print('Task complete!')

print('Company Name:', len(companies['Company Name']))
print('CIK:', len(companies['CIK']))
print('Form:', len(companies['Form']))
print('Filing Date:', len(companies['Filing Date']))
print('URL:', len(companies['URL']))

print('\n')
seconds = (time.time() - startTime)
seconds = int(seconds)
execution_time = datetime.timedelta(seconds=seconds)
print('Execution time (hours, minutes, seconds):', execution_time)

--------------------------------------------------
Year: 2011 --- Processing...
Year: 2011 --- Complete!
--------------------------------------------------
Year: 2012 --- Processing...
Year: 2012 --- Complete!
--------------------------------------------------
Year: 2013 --- Processing...
Year: 2013 --- Complete!
--------------------------------------------------
Year: 2014 --- Processing...
Year: 2014 --- Complete!
--------------------------------------------------
Year: 2015 --- Processing...
Year: 2015 --- Complete!
--------------------------------------------------
Year: 2016 --- Processing...
Year: 2016 --- Complete!
--------------------------------------------------
Year: 2017 --- Processing...
Year: 2017 --- Complete!
--------------------------------------------------
Year: 2018 --- Processing...
Year: 2018 --- Complete!


Task complete!
Company Name: 26585
CIK: 26585
Form: 26585
Filing Date: 26585
URL: 26585


Execution time (hours, minutes, seconds): 3:19:54


## 1994-2002

In [10]:
companies_1994_2002 = companies

df = pd.DataFrame(companies_1994_2002, dtype = str)
display(df.head(15))
display(df.info())

df.to_excel('1994-2002.xlsx')

## 2003 -2010

In [15]:
companies_2003_2010 = companies

df = pd.DataFrame(companies_2003_2010, dtype = str)
display(df.head(15))
display(df.info())

df.to_excel('2003-2010.xlsx')

,Company Name,CIK,Form,Filing Date,URL
0,21ST CENTURY HOLDING CO,0001069996,S-1/A,11/04/2003,https://www.sec.gov//Archives/edgar/data/10699...
1,21ST CENTURY HOLDING CO,0001069996,S-1,09/12/2003,https://www.sec.gov//Archives/edgar/data/10699...
2,2ND SWING INC,0001098044,S-1/A,08/12/2003,https://www.sec.gov//Archives/edgar/data/10980...
3,2ND SWING INC,0001098044,S-1,07/17/2003,https://www.sec.gov//Archives/edgar/data/10980...
4,2ND SWING INC,0001098044,S-1/A,06/02/2003,https://www.sec.gov//Archives/edgar/data/10980...
5,3D SYSTEMS CORP,0000910638,S-1,08/29/2003,https://www.sec.gov//Archives/edgar/data/91063...
6,99 CENT STUFF INC,0001176435,S-1/A,11/26/2003,https://www.sec.gov//Archives/edgar/data/11764...
7,99 CENT STUFF INC,0001176435,S-1/A,11/19/2003,https://www.sec.gov//Archives/edgar/data/11764...
8,99 CENT STUFF INC,0001176435,S-1/A,10/29/2003,https://www.sec.gov//Archives/edgar/data/11764...
9,99 CENT STUFF INC,0001176435,S-1/A,10/10/2003,https://www.sec.gov//Archives/edgar/data/11764...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26179 entries, 0 to 26178
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company Name  26179 non-null  object
 1   CIK           26179 non-null  object
 2   Form          26179 non-null  object
 3   Filing Date   26179 non-null  object
 4   URL           26179 non-null  object
dtypes: object(5)
memory usage: 1022.7+ KB


None

## 2011-2018

In [6]:
companies_2011_2018 = companies

df = pd.DataFrame(companies_2011_2018, dtype = str)
display(df.head(15))
display(df.info())

df.to_excel('2011-2018.xlsx')

,Company Name,CIK,Form,Filing Date,URL
0,1295728 Alberta ULC,0001391879,S-1/A,08/19/2011,https://www.sec.gov//Archives/edgar/data/15224...
1,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,07/01/2011,https://www.sec.gov//Archives/edgar/data/38723...
2,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,07/01/2011,https://www.sec.gov//Archives/edgar/data/38723...
3,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,06/07/2011,https://www.sec.gov//Archives/edgar/data/38723...
4,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,06/07/2011,https://www.sec.gov//Archives/edgar/data/38723...
5,1st FRANKLIN FINANCIAL CORP,0000038723,S-1,04/22/2011,https://www.sec.gov//Archives/edgar/data/38723...
6,1st FRANKLIN FINANCIAL CORP,0000038723,S-1,04/22/2011,https://www.sec.gov//Archives/edgar/data/38723...
7,"22nd Century Group, Inc.",0001347858,S-1/A,09/20/2011,https://www.sec.gov//Archives/edgar/data/13478...
8,"22nd Century Group, Inc.",0001347858,S-1/A,09/12/2011,https://www.sec.gov//Archives/edgar/data/13478...
9,"22nd Century Group, Inc.",0001347858,S-1/A,09/06/2011,https://www.sec.gov//Archives/edgar/data/13478...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26585 entries, 0 to 26584
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company Name  26585 non-null  object
 1   CIK           26585 non-null  object
 2   Form          26585 non-null  object
 3   Filing Date   26585 non-null  object
 4   URL           26585 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


None

## Using Pandas

In [24]:
df1 = pd.read_excel('1994-2002.xlsx')
df2 = pd.read_excel('2003-2010.xlsx')
df3 = pd.read_excel('2011-2018.xlsx')
del df1['Unnamed: 0']
del df2['Unnamed: 0']
del df3['Unnamed: 0']
display(df1)
display(df2)
display(df3)

,Company Name,CIK,Form,Filing Date,URL
0,ACME METALS INC /DE/,0000883702,S-1/A,08/04/1994,https://www.sec.gov//Archives/edgar/data/88370...
1,ACME METALS INC /DE/,0000883702,S-1/A,07/26/1994,https://www.sec.gov//Archives/edgar/data/88370...
2,ACME METALS INC /DE/,0000883702,S-1/A,07/14/1994,https://www.sec.gov//Archives/edgar/data/88370...
3,ACME METALS INC /DE/,0000883702,S-1,06/10/1994,https://www.sec.gov//Archives/edgar/data/88370...
4,ALCO STANDARD CORP,0000003370,S-1,12/19/1994,https://www.sec.gov//Archives/edgar/data/3370/...
...,...,...,...,...,...
22642,ZILOG INC,0000319450,S-1/A,11/05/2002,https://www.sec.gov//Archives/edgar/data/31945...
22643,ZILOG INC,0000319450,S-1,08/22/2002,https://www.sec.gov//Archives/edgar/data/31945...
22644,ZS ACQUISITION INC,0001174222,S-1/A,05/31/2002,https://www.sec.gov//Archives/edgar/data/11740...
22645,ZYMOGENETICS INC,0001129425,S-1/A,01/31/2002,https://www.sec.gov//Archives/edgar/data/11294...


,Company Name,CIK,Form,Filing Date,URL
0,21ST CENTURY HOLDING CO,0001069996,S-1/A,11/04/2003,https://www.sec.gov//Archives/edgar/data/10699...
1,21ST CENTURY HOLDING CO,0001069996,S-1,09/12/2003,https://www.sec.gov//Archives/edgar/data/10699...
2,2ND SWING INC,0001098044,S-1/A,08/12/2003,https://www.sec.gov//Archives/edgar/data/10980...
3,2ND SWING INC,0001098044,S-1,07/17/2003,https://www.sec.gov//Archives/edgar/data/10980...
4,2ND SWING INC,0001098044,S-1/A,06/02/2003,https://www.sec.gov//Archives/edgar/data/10980...
...,...,...,...,...,...
26174,SPS COMMERCE INC,0001092699,S-1/A,03/30/2010,https://www.sec.gov//Archives/edgar/data/10926...
26175,SPS COMMERCE INC,0001092699,S-1/A,03/05/2010,https://www.sec.gov//Archives/edgar/data/10926...
26176,SPS COMMERCE INC,0001092699,S-1/A,02/12/2010,https://www.sec.gov//Archives/edgar/data/10926...
26177,SPS COMMERCE INC,0001092699,S-1/A,01/11/2010,https://www.sec.gov//Archives/edgar/data/10926...


,Company Name,CIK,Form,Filing Date,URL
0,1295728 Alberta ULC,0001391879,S-1/A,08/19/2011,https://www.sec.gov//Archives/edgar/data/15224...
1,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,07/01/2011,https://www.sec.gov//Archives/edgar/data/38723...
2,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,07/01/2011,https://www.sec.gov//Archives/edgar/data/38723...
3,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,06/07/2011,https://www.sec.gov//Archives/edgar/data/38723...
4,1st FRANKLIN FINANCIAL CORP,0000038723,S-1/A,06/07/2011,https://www.sec.gov//Archives/edgar/data/38723...
...,...,...,...,...,...
26580,"Zscaler, Inc.",0001713683,S-1,02/16/2018,https://www.sec.gov//Archives/edgar/data/17136...
26581,ZUORA INC,0001423774,S-1MEF,04/11/2018,https://www.sec.gov//Archives/edgar/data/14237...
26582,ZUORA INC,0001423774,S-1/A,04/10/2018,https://www.sec.gov//Archives/edgar/data/14237...
26583,ZUORA INC,0001423774,S-1/A,04/02/2018,https://www.sec.gov//Archives/edgar/data/14237...


In [25]:
print('BEFORE', '-'*50)
display(df1['Form'].value_counts())
display(df2['Form'].value_counts())
display(df3['Form'].value_counts())

# As previously we remove extra type of values 
# leaving only S-1/A, S-1, and S-1MEF
values = ['S-3/A', 'S-3', 'POS462B', 'X-17A-5', 
          'D/A', 'D', 'N-8F ORDR', '497J', 'N-8F', 
          'N-8F NTC', '485APOS', 'FOCUSN', '497', 
          'PRES14A','DEFS14A', 'DEFA14A', 'N-30D', 
          'NSAR-B', 'POS AM', 'NSAR-A', '24F-2NT', 
          '485B24E', 'N-14AE']
df1 = df1[df1.Form.isin(values) == False]
df2 = df2[df2.Form.isin(values) == False]
df3 = df3[df3.Form.isin(values) == False]

print('AFTER', '-'*50)
display(df1['Form'].value_counts())
display(df2['Form'].value_counts())
display(df3['Form'].value_counts())

BEFORE --------------------------------------------------


S-1/A      15932
S-1         5883
S-1MEF       483
S-3/A        123
497           88
PRES14A       41
DEFS14A       41
DEFA14A       13
S-3           10
POS462B        9
N-30D          5
NSAR-B         4
NSAR-A         3
POS AM         3
497J           3
24F-2NT        2
485B24E        2
485APOS        1
N-14AE         1
Name: Form, dtype: int64

S-1/A        17918
S-1           7765
S-1MEF         333
S-3/A          142
S-3             12
X-17A-5          4
N-8F ORDR        1
FOCUSN           1
N-8F NTC         1
N-8F             1
D/A              1
Name: Form, dtype: int64

S-1/A     18379
S-1        7599
S-1MEF      590
S-3/A         9
S-3           3
D/A           3
D             2
Name: Form, dtype: int64

AFTER --------------------------------------------------


S-1/A     15932
S-1        5883
S-1MEF      483
Name: Form, dtype: int64

S-1/A     17918
S-1        7765
S-1MEF      333
Name: Form, dtype: int64

S-1/A     18379
S-1        7599
S-1MEF      590
Name: Form, dtype: int64

Save three dataframes into signle excel file, separated by worksheets

In [26]:
# Save into excel file
with pd.ExcelWriter("Companies1994-2018.xlsx") as writer:
    df1.to_excel(writer, sheet_name = "1994-2002", index = False)
    df2.to_excel(writer, sheet_name = "2003-2010", index = False)
    df3.to_excel(writer, sheet_name = "2011-2018", index = False)

print('Save complete')

Save complete
